Risk rates report (for securities at this moment). This report is made for indication of abnormal and top risk rates changes, changes of number of margin/non-margin financial instruments.

Отчёт по ставкам риска финансовых инструментов (для акций). Отчёт нужен для подсвечивания аномальных и наибольших изменений ставок риска по финансовым инструментам, изменения количества маржинальных/немаржинальных инструментов. 

In [ ]:
# Load whole financial_instrument table (100-150k lines)
# Выгрузка всей таблицы financial_instrument

import pandas as pd
import math
import numpy as np
from datetime import timedelta
import tinkoff as tf

query = "SELECT * FROM prod_v_dds.financial_instrument ORDER BY valid_from_dttm ASC"
with tf.gp_connect('gp_workaround') as connection_obj:
    df = tf.gp_to_df(query, 'gp_workaround')
df.head()

In [ ]:
# Working with data types + sorting data by name of fin.instrument & validation date (from)
# Generating dataframe without n/a values of risk rates

# Приведение типов + сортировка данных по имени инструмента, дате записи. 
# Запись датафрейма без пустых значений по ставкам риска.

df['issure_dt'] = pd.to_datetime(df['issure_dt'])
df['placing_dt'] = pd.to_datetime(df['placing_dt'])
df = df.sort_values(['financial_instrument_nm', 'valid_from_dttm'])
df_cleared = df.dropna(subset=['margin_short_rate', 'margin_long_rate'])
df_cleared.head()

In [ ]:
# Creating columns with previous values of risk rates. Separate flag for short_rate = 9999*
# Создание колонок с предыдущими значениями ставок риска. Отдельный флаг для ставок риска шорт = 9999

import numpy as np
df_cleared['prev_margin_short_rate'] = np.nan
df_cleared['prev_margin_long_rate'] = np.nan
df_cleared['short_rate_was_9999'] = 0
print(df_cleared.dtypes)

# *Short rate equal 9999 is a feature of technical realization in early times. It's the same as 1.

In [ ]:
# Set risk_rate_short=9999 to 1
# Перезадаём ставки риска шорт, равные 9999.

df_cleared.loc[(df_cleared['margin_short_rate'] == 9999), 'short_rate_was_9999'] = 1
df_cleared.loc[(df_cleared['margin_short_rate'] == 9999), 'margin_short_rate'] = 1
df_cleared.head(1000)


isins = df_cleared['isin'].unique()

for isin in isins:
    df_cleared.loc[(df_cleared['isin'] == isin), 'prev_margin_short_rate'] = df_cleared[df_cleared['isin'] == isin]['margin_short_rate'].shift(1)
    df_cleared.loc[(df_cleared['isin'] == isin), 'prev_margin_long_rate'] = df_cleared[df_cleared['isin'] == isin]['margin_long_rate'].shift(1)

df_cleared['margin_short_delta'] = df_cleared['margin_short_rate'] - df_cleared['prev_margin_short_rate']
df_cleared['margin_long_delta'] = df_cleared['margin_long_rate'] - df_cleared['prev_margin_long_rate']

In [ ]:
# Lists of value changes - for all instruments & for separate groups (without n/a values)
# Списки с величинами изменений - для всех инструментов и по отдельным группам (без n/a значений)

margin_short_delta_all = list(df_cleared['margin_short_delta'].dropna().reset_index(drop=True))
margin_long_delta_all = list(df_cleared['margin_long_delta'].dropna().reset_index(drop=True))
margin_short_delta_sec = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']['margin_short_delta']\
                              .dropna().reset_index(drop=True))
margin_long_delta_sec = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']['margin_long_delta']\
                              .dropna().reset_index(drop=True))
margin_short_delta_bnd = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'BND']['margin_short_delta']\
                              .dropna().reset_index(drop=True))
margin_long_delta_bnd = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'BND']['margin_long_delta']\
                              .dropna().reset_index(drop=True))
margin_short_delta_der = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'DER']['margin_short_delta']\
                              .dropna().reset_index(drop=True))
margin_long_delta_der = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'DER']['margin_long_delta']\
                              .dropna().reset_index(drop=True))
margin_short_delta_oth = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'OTH']['margin_short_delta']\
                              .dropna().reset_index(drop=True))
margin_long_delta_oth = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'OTH']['margin_long_delta']\
                              .dropna().reset_index(drop=True))
margin_short_delta_nan = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'nan']['margin_short_delta']\
                              .dropna().reset_index(drop=True))
margin_long_delta_nan = list(df_cleared.loc[df_cleared['instrument_type_cd'] == 'nan']['margin_long_delta']\
                              .dropna().reset_index(drop=True))

In [ ]:
# Lists of value changes (abs taken) of risk_rates_short for securities
# What should we do with 0 values? At this moment: take them off, 
# because updating data doesn't always concern (both) risk rates, but always record to a new line

# Формируем списки с величинами изменений (по модулю) ставок шорт по акциям
# Что делать с нулями? На данный момент: нули нужно убрать, т.к. обновления данных по фин.инструменту 
# не всегда касается (обеих) ставок риска, но всегда записывается в новую строку
margin_short_delta_sec = [i for i in margin_short_delta_sec if i != 0]
margin_short_delta_sec = [abs(i) for i in margin_short_delta_sec]
margin_short_delta_sec

In [ ]:
# Calculating 99, 95, 90 percentile risk rate change for securities
# Рассчитываем 99, 95, 90-й перцентиль изменения ставок риска шорт по акциям

delta_short_sec_perc_99 = np.quantile(margin_short_delta_sec, 0.99)
delta_short_sec_perc_95 = np.quantile(margin_short_delta_sec, 0.95)
delta_short_sec_perc_90 = np.quantile(margin_short_delta_sec, 0.9)
print(f"99: {delta_short_sec_perc_99}\n95: {delta_short_sec_perc_95}\n90: {delta_short_sec_perc_90}")

In [ ]:
# Short risk rate change, fit in percentile, to date (today)
# Изменения ставок риска шорт, вошедшие в перцентиль, в текущую дату

#Вывести изменения ставок риска, вошедшие в перцентиль, в эту дату (SEC, 90, 11.04.2023)
#Отбор по: типу инструмента, вхождению изменения ставки в перцентиль, интересующей дате

today_date = str(datetime.now().date())
short_perc_sec = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
                            .loc[abs(df_cleared['margin_short_delta']) > delta_short_sec_perc_90]\
                            .loc[df_cleared['valid_from_dttm'] == today_date]
short_perc_sec

#Спойлер: изменений  ставок риска шорт на конкретный день по одному виду фин.инстр. мало, 
#более того, изменения, входящие в рассчитанные перцентили, наблюдаются далеко не каждый день

In [ ]:
# Long risk rate changes, fit in percentile, to date (today)
# Изменения ставок риска лонг, вошедшие в перцентиль, в текущую дату

margin_long_delta_sec = [i for i in margin_long_delta_sec if i != 0]
margin_long_delta_sec = [abs(i) for i in margin_long_delta_sec]

delta_long_sec_perc_99 = np.quantile(margin_long_delta_sec, 0.99)
delta_long_sec_perc_95 = np.quantile(margin_long_delta_sec, 0.95)
delta_long_sec_perc_90 = np.quantile(margin_long_delta_sec, 0.9)
print(f"{delta_long_sec_perc_99}\n{delta_long_sec_perc_95}\n{delta_long_sec_perc_90}")

long_perc_sec = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
                            .loc[abs(df_cleared['margin_long_delta']) > delta_long_sec_perc_90]\
                            .loc[df_cleared['valid_from_dttm'] == today_date]
long_perc_sec

In [ ]:
# Table of abnormal risk rate change (long and short for securities)
# Таблицы аномальных изменений ставок риска шорт и лонг (по акциям)

from tabulate import tabulate

header = ["Полное наименование инструмента", "Тип инструмента", "ISIN", "Ставка шорт/лонг", "Стало", "Было"]
table = []
abnormal_table = ''

if short_perc_sec.empty:
    abnormal_table += f"<p>Аномальных изменений <i>ставок риска шорт</i> по акциям не выявлено.</p>"
else:
    for index, row in short_perc_sec.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'шорт', row['margin_short_rate'], row['prev_margin_short_rate']])

if long_perc_sec.empty:
    abnormal_table += f"<p>Аномальных изменений <i>ставок риска лонг</i> по акциям не выявлено.</p>"
else:
    for index, row in long_perc_sec.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'лонг', row['margin_long_rate'], row['prev_margin_long_rate']])

# print(tabulate(table, header, tablefmt="html"))
abnormal_table += tabulate(table, header, tablefmt="html")

In [ ]:
# Take top-5 biggest change (in both directions) of risk rates (securities)
# Отберём по 5 крупнейших изменений (в обе стороны) ставок риска по акциям:

top5_incr_short_rate = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
        .loc[df_cleared['valid_from_dttm'] == today_date]\
        .loc[df_cleared['margin_short_delta'] > 0].sort_values('margin_short_delta', ascending=False).head(5)
top5_decr_short_rate = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
        .loc[df_cleared['valid_from_dttm'] == today_date]\
        .loc[df_cleared['margin_short_delta'] < 0].sort_values('margin_short_delta', ascending=True).head(5)

top5_incr_long_rate = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
        .loc[df_cleared['valid_from_dttm'] == today_date]\
        .loc[df_cleared['margin_long_delta'] > 0].sort_values('margin_long_delta', ascending=False).head(5)
top5_decr_long_rate = df_cleared.loc[df_cleared['instrument_type_cd'] == 'SEC']\
        .loc[df_cleared['valid_from_dttm'] == today_date]\
        .loc[df_cleared['margin_long_delta'] < 0].sort_values('margin_long_delta', ascending=True).head(5)

In [ ]:
# Tables of top risk rate change (long and short, for securities, both directions)
# Таблицы топ изменений ставок риска шорт и лонг (по акциям) в обе стороны

header = ["Полное наименование инструмента", "Тип инструмента", "ISIN", "Ставка шорт/лонг", "Стало", "Было", "Дельта"]
table = []
top5rates_table = ''

if top5_incr_short_rate.empty:
    top5rates_table += "<p><i>Роста ставок риска шорт</i> по акциям не произошло.</p>"
else:    
    for index, row in top5_incr_short_rate.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'шорт', row['margin_short_rate'],\
                        row['prev_margin_short_rate'], row['margin_short_delta']])

if top5_decr_short_rate.empty:
    top5rates_table += "<p><i>Падений ставок риска шорт</i> по акциям не произошло.</p>"
else:
    for index, row in top5_decr_short_rate.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'шорт', row['margin_short_rate'],\
                        row['prev_margin_short_rate'], row['margin_short_delta']])

if top5_incr_long_rate.empty:
    top5rates_table += "<p><i>Роста ставок риска лонг</i> по акциям не произошло.</p>"
else:                        
    for index, row in top5_incr_long_rate.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'лонг', row['margin_long_rate'],\
                        row['prev_margin_long_rate'], row['margin_long_delta']])

if top5_decr_long_rate.empty:
    top5rates_table += "<p><i>Падений ставок риска лонг</i> по акциям не произошло.</p>"
else:
    for index, row in top5_decr_long_rate.iterrows():
        table.append([row['financial_instrument_nm'], row['instrument_type_cd'], row['isin'], 'лонг', row['margin_long_rate'],\
                        row['prev_margin_long_rate'], row['margin_long_delta']])

top5rates_table += tabulate(table, header, tablefmt="html")
top5rates_table 


Блок статистики: кол-во маржинальных и не маржинальных инструментов

In [ ]:
# Statistic block (for securities, for today)
# 1) total number of sec,
# 2) count margin instruments,
# 3) number of margin instr. change,
# 4) number of sec with risk rate change

# Блок статистики (пока для акций и на актуальную дату): 
# 1) подсчитать общее число бумаг, 
# 2) подсчитать маржинальные, 
# 3) изменение по кол-ву маржинальных ЦБ,
# 4)кол-во бумаг, по которым изменились ставки риска

# Total number of every kind of instruments (unique financial_instrument_nm in actual records counted)
# Общее число бумаг по каждому типу инструментов (подсчитаны уникальные financial_instrument_nm по действующим записям)
max_date = datetime(5999, 1, 1, 0, 0)
full_amount_active_instruments = df.loc[df['valid_to_dttm'] == max_date]\
                                    .groupby('instrument_type_cd', as_index=False)\
                                    .agg({'financial_instrument_nm': 'nunique'})\
                                    .rename(columns={'financial_instrument_nm': 'number_of_instruments'})
full_amount_active_sec = int(full_amount_active_instruments.loc[full_amount_active_instruments['instrument_type_cd'] == 'SEC']['number_of_instruments'].iloc[0])

In [ ]:
# Number of margin instr., total and securities seperately (actual)
# Посчитаем кол-во маржинальных бумаг, всего и конкретно акций (актуальных)

margin_instruments = df[((df['margin_short_rate'].isna() == False) & (df['margin_long_rate'].isna() == False))\
    & ((df['margin_short_rate'] != 1) & (df['margin_long_rate'] != 1))]
margin_instruments[margin_instruments['valid_to_dttm'] == max_date]

margin_instruments_by_category = margin_instruments[margin_instruments['valid_to_dttm'] == max_date]\
        .groupby('instrument_type_cd', as_index=False).agg({'financial_instrument_nm': 'nunique'})\
        .rename(columns={'financial_instrument_nm': 'number_of_margin'})
number_margin_instruments_sec = int(margin_instruments_by_category[margin_instruments_by_category['instrument_type_cd'] == 'SEC']['number_of_margin'])

In [ ]:
# Number of non-margin instruments (+ securities separately)
# Считаем немаржинальные инструменты (в т.ч. акций)

non_margin_instruments = df[((df['margin_short_rate'].isna()) & (df['margin_long_rate'].isna()))\
    | ((df['margin_short_rate'] == 1) & (df['margin_long_rate'] == 1))]
non_margin_instruments_by_category = non_margin_instruments[non_margin_instruments['valid_to_dttm'] == max_date]\
        .groupby('instrument_type_cd', as_index=False).agg({'financial_instrument_nm': 'nunique'})\
        .rename(columns={'financial_instrument_nm': 'number_of_margin'})
number_non_margin_instruments_sec = int(non_margin_instruments_by_category[non_margin_instruments_by_category['instrument_type_cd'] == 'SEC']['number_of_margin'])

In [ ]:
# Number of margin instruments
# Считаем маржинальные инструменты

today = datetime.now()
yesterday = today - timedelta(days=1)

margin_instruments_today = df[((df['margin_short_rate'].isna() == False) & (df['margin_long_rate'].isna() == False))\
    & ((df['margin_short_rate'] != 1) & (df['margin_long_rate'] != 1))\
    & (df['valid_from_dttm'] <= today) & (df['valid_to_dttm'] == max_date)]

margin_instruments_yesterday = df[((df['margin_short_rate'].isna() == False) & (df['margin_long_rate'].isna() == False))\
    & ((df['margin_short_rate'] != 1) & (df['margin_long_rate'] != 1))\
    & (df['valid_from_dttm'] <= yesterday) \
    & (((df['valid_to_dttm'] >= yesterday) & (df['valid_to_dttm'] <= today)) | (df['valid_to_dttm'] == max_date))]

total_margin_sec_today = margin_instruments_today[margin_instruments_today['instrument_type_cd'] == 'SEC'].financial_instrument_nm.nunique()
total_margin_sec_yesterday = margin_instruments_yesterday[margin_instruments_yesterday['instrument_type_cd'] == 'SEC'].financial_instrument_nm.nunique()

In [ ]:
statement_margin_instruments_change_sec = ''
total_margin_sec_change = total_margin_sec_today - total_margin_sec_yesterday

if total_margin_sec_change == 0:
    statement_margin_instruments_change_sec = 'Со вчерашнего дня количество акций с возможностью маржинальной торговли не изменилось.'
elif total_margin_sec_change > 0:
    statement_margin_instruments_change_sec = 'Со вчерашнего дня количество акций с возможностью маржинальной торговли увеличилось на ' + str(total_margin_sec_change) + '.'
else:
    statement_margin_instruments_change_sec = 'Со вчерашнего дня количество акций с возможностью маржинальной торговли уменьшилось на ' + str(abs(total_margin_sec_change)) + '.'

In [ ]:
stat_block = f"<p><i>По состоянию на {today.date()}:</i><br><br>Общее количество акций: {full_amount_active_sec}.<br>\
Количество маржинальных акций: {number_margin_instruments_sec} \
({round(number_margin_instruments_sec * 100 / full_amount_active_sec, 2)}% от общего числа).<br>\
{statement_margin_instruments_change_sec}<br></p>"
stat_block

In [ ]:
# Generate report in html
# Формирование отчёта в html

report = '''<html>\
<head>\
<style type="text/css">\
    p {\
        margin-top: 0px;\
        margin-bottom: 12px;\
    }\
    TABLE {\
        border-collapse: collapse;\
    }\
    TD, TH {\
        border: 1px solid black;\
        padding: 4px;\
    }\
</style>\
</head>\
<body>'''
report += f"{stat_block}\
<p><i>{today.date()} произошли следующие аномальные изменения ставок риска:\
(больше, чем на {str(round(delta_short_sec_perc_90,4))} для ставок шорт, больше, чем на {str(round(delta_long_sec_perc_90,4))} для ставок лонг):</i></p>\
{abnormal_table}\
<p><i>{today.date()} крупнейшие изменения (как рост, так и уменьшение) ставок риска наблюдались у следующих инструментов:</i></p>\
{top5rates_table}<br>\
</body>"

In [ ]:
# Sending report
# Отправка отчёта

import tinkoff as tf
subject = 'Отчёт по ставкам риска (только акции, beta)'
addressee = ['i.baryshnikov@tinkoff.ru', 'e.v.varlamova@tinkoff.ru']
cc = 'r.iskyandyarov@tinkoff.ru'
tf.send_email(subject, report, addressee, cc=cc, files = None, from_user = True)